PART3:

Step1.Load and Store data

1.We stored the input data in List[List[Str]]

2.1st List includ all sentences in a file

3.2nd List includ all words in one sentence

4.Str is one of the world in each sentence

In [9]:
from pathlib import Path
FOLDER = Path('./MachineLearning-Project/')


#EN
EN = FOLDER/'EN(2)'
Train_EN = 'ENtrain'
dev_x_EN = 'ENdev.in'
dev_y_EN = 'ENdev.out'
ENpart2_out = './EN(2)/dev.p2.out'
ENpart3_out = './EN(2)/dev.p3.out'

#CN
CN = FOLDER/'CN'
Train_CN = 'CNtrain'
dev_x_CN = 'CNdev.in'
dev_y_CN = 'CNdev.out'
CNpart2_out = './CN/dev.p2.out'
CNpart3_out = './CN/dev.p3.out'

#SG
SG = FOLDER/'SG(1)'
Train_SG = 'SGtrain'
dev_x_SG = 'SGdevc.in'
dev_y_SG = 'SGdev.out'
SGpart2_out = './SG(1)/dev.p2.out'
SGpart3_out = './SG(1)/dev.p5.out'
EVALscript ='evalResult.py'


Step2.

Run HMM model and got Results using HMM model that we created
Here is the HMM Algorithem that we defined


1.At the beginning and end of sentence,we add <SOS> and <EOS> to show start and End.
    
2.We set rows are initial state

3.We set columns are the state trasitted to.(next state)

    
4.We iterate each sentence for each file.
    
5.There is a $(|T|+1)\times(|T|+1)$ matrix to store count value. 
    
6.We calculate the probabilities by dividing each entry with the sum of correspodning row.

7.Write an Viterbi Algorithm, implement two matrix.
    
    parent matrix(size $n\times(|T|+1)$)
    score matrix (size $n\times(|T|+1)$)
    
8.set optimal path, which is restored by finding the argmax among $|T|$ tags in previous position


9.Set k=1

In [10]:
#Import necessary library
import warnings
warnings.filterwarnings('ignore')
import os
from pathlib import Path
from collections import Counter, defaultdict
import numpy as np

In [11]:
#Set index
def get_index_number(max_arg, index_value, kvalue):
    count = 0
    for o in max_arg:
        if o == index_value:
            return count
        if o//kvalue == index_value//kvalue:
            count += 1


#Get data from training file 
def get_data(nameof_trainingfile):
    with open(nameof_trainingfile) as f:
        lineslis = f.readlines()
    temx, temy = [], []
    x,y = [],[]
    
    for sen in lineslis:
        if len(sen) == 1:
            assert(len(temx) == len(temy))
            x.append(temx)
            y.append(temy)
            temx, temy = [], []
            continue
            
        xv,yv = sen.split()
        temx.append(xv)
        temy.append(yv)
    if len(temx) !=0:
        x.append(temx)
        y.append(temy)
    assert(len(x)==len(y))
    
    return x,y

#Testing command 
#get_data(SG_train) 
#get_data()return two list, one is include all words for each sentence,
#the other(y) is include all tags for each sentence
#words,labels = get_data(SG_train)
#print(words)
#print(labels)

In [12]:
#Get Test data
#x is the nested list of string
#x_int is the nested list of integer
def get_tdata(file, w2index):
    with open(file) as f:
        lineslis =f.readlines()
    x =[]
    temx =[]
    for sen in lineslis:
        if len(sen.strip())== 0:
            x.append(temx)
            temx  = []
            continue
        xv = sen.split()
        temx.append(xv[0])
        
    if len(temx) != 0:
        x.append(temx)
    x_int = [[w2index[oo] for oo in o] for o in x ]
    return x, x_int

In [25]:
#Define a HMM class
#Find emission parameters & Transition parameters
#Run viteribi algorithem ---- Encode & Decode 

class HMM:
    def __init__(self, tf):
        # read data
        # create vocab
        # Set tagslabel at satrt state and final state SOS &EOS
        
        self.words, self.labels = get_data(tf)
        self.labeltags = list(set([oo for o in self.labels for oo in o])) + ['SOS', 'EOS']
        self.tag2index = {o:i for i,o in enumerate(self.labeltags)}
        vocab_count = Counter([oo for o in self.words for oo in o])
        self.vocab = [o for o, v in dict(vocab_count).items() if v>=3] + ['#UNK#']
        self.word2index = defaultdict(int)
        
        for i,o in enumerate(self.vocab): self.word2index[o] = i+1
        self.x = [[self.word2index[oo] for oo in o] for o in self.words]
        self.y = [[self.tag2index[oo] for oo in o] for o in self.labels]
    
    def train(self):
        self.transition_para()
        self.emission_para()
    
    def transition_para(self):
        """
        Step : 
        1. Rows: transition from current state : (v1, v2, ..., 'SOS')
        2. Cols: to next state :   (v1, v2, ..., 'EOS')
        3. two new state sos , eos 
        4. START transition
           Set empty array, add value in 
        5. label tags transition from position 0 to len(yvalue)-2
        6. STOP transition
        
        """
        transition = np.zeros((len(self.labeltags)-1, len(self.labeltags)-1))
        
        for yvalue in self.y: 
            transition[-1, yvalue[0]] += 1 
            for i in range(len(yvalue)-1): 
                transition[yvalue[i], yvalue[i+1]] += 1
            transition[yvalue[-1], -1] += 1 
        transition = transition/np.sum(transition, axis=1)
        self.transition = transition
    
    
    def emission_para(self):
        
        emission_matrix = np.zeros((len(self.vocab), len(self.labeltags)))
        
        
        
        fx = [oo for o in self.x for oo in o]
        fy = [oo for o in self.y for oo in o]
        
        
        for xvalue, yvalue in zip(fx,fy):
            emission_matrix[xvalue, yvalue] += 1
    
        yc = np.zeros(len(self.labeltags))
        
        for yvalue in fy:
            yc[yvalue] += 1
        
        emission_matrix  = emission_matrix / yc[None, :]
        np.nan_to_num(emission_matrix , 0)
        self.emission = emission_matrix 
    
    def predict_top_k(self, inpfile, outfile, k=1):
        
        assert hasattr(self, 'transition') and hasattr(self, 'emission'), "run self.train() first"
        with open(outfile, 'w') as f:
            words, dev_x = get_tdata(inpfile, self.word2index)
            slis = []
            for i, (wsvalue,o) in enumerate(zip(words, dev_x)):
                path, logmax= self.viterbi(o, k)
                slis.append(logmax)
                for w, p in zip(wsvalue, path):
                    f.write(w + ' ' + self.labeltags[p] + '\n')
                f.write('\n')
        return
    
    def viterbi(self, x, k=1):
        score = np.zeros( (len(x)+2, len(self.labeltags)-2, k) ) 
        argmax = np.zeros( (len(x)+2, len(self.labeltags)-2, k), dtype=np.int) 
        transition, emission = np.log(self.transition), np.log(self.emission)
        # initialization at j=1
        score[1, :, 0] = (transition[-1, :-1] + emission[x[0], :-2])
        score[1, :, 1:] = -np.inf
        # j=2, ..., n
        for j in range(2, len(x)+1): 
            for t in range(len(self.labeltags)-2):
                pi = score[j-1, :]  # (num_of_labeltags-2, 7)
                a = transition[:-1, t] # (num_of_labeltags-2,)
                b = emission[x[j-1], t] # (1,)
                previous_all_scores = ((pi + a[:, None])).flatten()
                topk = previous_all_scores.argsort()[-k:][::-1] # big to small
                argmax[j, t] = topk # big: 0, small: -1 # topk // k is the real argument
                score[j, t] = previous_all_scores[topk] + b

        # j=n+1 step
        pi = score[len(x)] # (num_of_labeltags-2, 7)
        a = transition[:-1, -1]
        argmax_stop = (pi + a[:,None]).flatten().argsort()[-k:][::-1] # big to small
        log_likelihood = np.min(pi+a[:,None])
        argmax = argmax[2:-1] # (len(x)-1, num_of_labeltags-2, 7)
        # decoding

        ## Initialize the last pointer backward
        temp_index = argmax_stop[-1] # range from 0 ~ k * ( len(labeltags) - 1 )
        temp_index_index = get_index_number(argmax_stop, temp_index, k) # range from 0 ~ k-1, next index in next tag
        temp_arg = temp_index // k # range from 0 ~ k-1, next tag index
        
        path = [argmax_stop[-1]//k] # initialize path as an array

        for i in range(len(argmax)-1, -1, -1):
            temp_index = argmax[i, temp_arg, temp_index_index]
            temp_index_index = get_index_number(argmax[i, temp_arg], temp_index, k)
            temp_arg = temp_index // k
            path.append(temp_arg)
        return path[::-1], log_likelihood 

Part3 Result

1.Run HMM Model

2.Run train() method 

3.Run Eval_Script to get the precision, recall and F score 

In [ ]:
#HMM MODEL -----Test SG
hmm = HMM(Train_SG)
hmm.train()
hmm.predict_top_k(dev_x_SG, SGpart3_out, k=1)


In [15]:
!python3 evalResult.py SGdev.out SGdev.p3.out


#Entity in gold data: 4301
#Entity in prediction: 3125

#Correct Entity : 1901
Entity  precision: 0.6083
Entity  recall: 0.4420
Entity  F: 0.5120

#Correct Sentiment : 1686
Sentiment  precision: 0.5395
Sentiment  recall: 0.3920
Sentiment  F: 0.4541


In [16]:
#HMM MODEL -----Test EN
hmm = HMM(Train_EN)
hmm.train()
hmm.predict_top_k(dev_x_EN, ENpart3_out, k=1)

In [17]:
!python3 evalResult.py Endev.out ENdev.p3.out


#Entity in gold data: 13179
#Entity in prediction: 12733

#Correct Entity : 10800
Entity  precision: 0.8482
Entity  recall: 0.8195
Entity  F: 0.8336

#Correct Sentiment : 10387
Sentiment  precision: 0.8158
Sentiment  recall: 0.7881
Sentiment  F: 0.8017


In [18]:
#HMM MODEL -----Test CN
hmm = HMM(Train_CN)
hmm.train()
hmm.predict_top_k(dev_x_CN, CNpart3_out, k=1)

In [19]:
!python3 evalResult.py CNdev.out CNdev.p3.out


#Entity in gold data: 700
#Entity in prediction: 311

#Correct Entity : 106
Entity  precision: 0.3408
Entity  recall: 0.1514
Entity  F: 0.2097

#Correct Sentiment : 71
Sentiment  precision: 0.2283
Sentiment  recall: 0.1014
Sentiment  F: 0.1405
